In [ ]:
using DataFrames, Distributions, ForwardDiff, Plots

In [ ]:
# Stochastic gradient descent, mixed logit.

df = readtable("../data/parsed_model_australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b
const n_simulations = 1000
const simulated_b = 5

srand(123456)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

head(df)

In [ ]:
mixed_logit = zeros(n_individuals, n_simulations)

function simulate()
    for i = 1:n_individuals, j = 1:n_simulations
        mixed_logit[i, j] = rand()
    end
end

simulate()

In [ ]:
function individual(θ::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function construct(γ::Vector, θ::Vector)
        return θ[1]+θ[2]*γ[1]
    end
    
    function probability(θ::Vector)
        logit = 0.0
        t = 0.0
        for k = 1:n_simulations
            β = []
            for j = 1:simulated_b-1
                push!(β, θ[j])
            end
            γ = [mixed_logit[i, k]]
            push!(β, construct(γ, θ[simulated_b:simulated_b+1]))
            for j = simulated_b+2:length(θ)
                push!(β, θ[j])
            end 
            c = utility(β, choice)
            for alternative in alternatives
                t += exp(utility(β, alternative)-c)
            end
            logit += 1/(1+t)
        end
        return logit/n_simulations
    end
    
    return probability
end

In [ ]:
function f(β::Vector)
    model = 0.0
    i = 1
    while i <= n_individuals
        logit = individual(β, i)
        model += log(logit(β))
        i += 1
    end
    return -model/n_individuals
end

In [ ]:
function g(x::Vector, i::Int64, j::Int64)
    t = zeros(length(x))
    for k = i:j
        logit = individual(x, k)
        t += (1/logit(x))*ForwardDiff.gradient(logit, x)
    end
    return -t/j
end

function g!(x::Vector, i::Int64, j::Int64, storage::Vector)
    s = g(x, i, j)
    storage[1:length(s)] = s[1:length(s)]
end

In [ ]:
function scores!(x::Vector, i::Int64, j::Int64, t::Matrix)
    for k = i:j
        logit = individual(x, k)
        t[:, k] = (1/logit(x))*ForwardDiff.gradient(logit, x)
    end
    return -t
end

In [ ]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int)
    χ::Float64 = 0.1
    θ::Float64 = 0.5
    if (k == kmax) || (normg <= normg0*min(χ, normg0^θ))
        return true
    else
        return false
    end
end

In [ ]:
function TruncatedCG(g::Vector, H::Matrix, Δ::Float64)
    n = length(g)
    s = zeros(n)
    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g, v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ
    while stopCG(norm(g), normg0, k, n) == false
        weight = 1.0/n_individuals
        κ = 0.0
        for j = 1:size(H)[2]
            κ += (dot(d, H[:, j]))^2
        end
        κ *= weight
        if κ <= 0
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        α = gv/κ
        norm2s += α*(2*sMd+α*norm2d)
        if norm2s >= Δ2
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        s += α*d
        for j = 1:size(H)[2]
            g += weight*(α*dot(H[:, j], d)*H[:, j])
        end
        v = g
        newgv = dot(g, v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        k += 1
    end
    return s
end

In [ ]:
function shuffle!(df::DataFrame)
    for i = size(df, 1):-1:2
        j = rand(1:i)
        df[i, :], df[j, :] = df[j, :], df[i, :]
    end
end

In [ ]:
function graph()
    default(size = (700, 700), fc = :heat)
    β1, β2 = 0:0.05:1.0, 0:0.05:1.0
    z = Surface((β1, β2) -> f([β1, β2, 0, 0, 0, 0, 0]), β1, β2)
    surface(β1, β2, z, linealpha = 0.3)
end

In [ ]:
function sgd(f::Function, g::Function, Step::Function, ω0::Vector, η::Float64,
    batch::Int64 = n_individuals, tol::Float64 = 1e-8, record::Bool = true)
    k = 0
    ω = ω0
    n = length(ω)
    dfx = ones(n)
    scores = zeros(n, batch)
    nmax = 500
    change = 1.0
    if record
        fω = f(ω)
        iterates = [fω ω']
    end
    while change > tol && k < nmax
        shuffle!(df)
        x = ω
        for i = 1:batch:n_individuals
            g(x, i, batch, dfx)
            scores!(x, i, batch, scores)
            step = Step(dfx, scores, η)
            ω += step
        end
        if record
            fω = f(ω)
            iterates = [iterates; fω ω']
        end
        change = abs(f(ω)-f(x))
        k += 1
    end
    if record
        return ω, k, iterates
    else
        return ω, k
    end
end

In [ ]:
sol, epochs, iter = sgd(f, g!, TruncatedCG, [20.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0], 0.01)

sol, epochs

In [ ]:
k = [x = i for i = 1:length(iter[:, 1])]

Plots.plot(k, iter[:, 1], label = "f(ω)")

In [ ]:
graph()